# Experiment Tracing
Experiment Tracking helps figure out what works and what doesn't

## 0. Getting Setup

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms
from torchinfo import summary
from going_modular import data_setup, engine

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def set_seeds(seed: int=42):
    """Sets random seed for torch operations"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

## 1. Get Data

In [4]:
import os
import zipfile
from pathlib import Path
import requests

def download_data(source: str, destination: str, remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from the source and unzips to destination"""
    data_path = Path("data")
    image_path = data_path/destination
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists. skipping download...")
        target_file = Path(source).name
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        target_file = Path(source).name
        with open(data_path/target_file,"wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)
        with zipfile.ZipFile(data_path/target_file,"r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...")
            zip_ref.extractall(image_path)
        if remove_source:
            os.remove(data_path/target_file)
    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] data\pizza_steak_sushi directory exists. skipping download...


WindowsPath('data/pizza_steak_sushi')

## 2. Create Datasets and DataLoaders

In [5]:
train_dir = image_path/"train"
test_dir = image_path/"test"
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
print(f"Automatically created transforms: {automatic_transforms}")
train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=automatic_transforms,
                                                                               batch_size=32)
train_dataloader, test_dataloader

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x1b469f48eb0>,
 <torch.utils.data.dataloader.DataLoader at 0x1b469f49d20>)

## 3. Getting a pretrained model, freezing the base layers and changing the classifier head

In [6]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [7]:
summary(model, 
        input_size=(32,3,224,224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=10,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape Output Shape Param #    Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224] [32, 1000] --         True
├─Sequential (features)                                      [32, 3, 224, 224] [32, 1280, 7, 7] --         True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224] [32, 32, 112, 112] --         True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224] [32, 32, 112, 112] 864        True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112] [32, 32, 112, 112] 64         True
│    │    └─SiLU (2)                                         [32, 32, 112, 112] [32, 32, 112, 112] --         --
│    └─Sequential (1)                                        [32, 32, 112, 112] [32, 16, 112, 112] --         True
│    │    └─MBConv (0)                                       [32, 32, 112, 112] [32, 16, 112, 112] 1,448

In [8]:
for param in model.features.parameters():
    param.requires_grad = False

set_seeds()
model.classifier = torch.nn.Sequential(nn.Dropout(p=0.2,inplace=True),
                                       nn.Linear(in_features=1280, out_features=len(class_names),bias=True)).to(device)


## 4. Train model and track results

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [11]:
from typing import Dict, List
from tqdm import tqdm
from going_modular.engine import train_step, test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device=device) -> Dict[str,List]:
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        print(f"Epoch: {epoch+1} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | test_loss: {test_loss:.4f} | test_acc: {test_acc:.4f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc},
                           global_step=epoch)
        writer.add_graph(model=model,
                         input_to_model=torch.randn(32,3,224,224).to(device))

    writer.close()
    return results

In [12]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0924 | train_acc: 0.3984 | test_loss: 0.9132 | test_acc: 0.5398


 20%|████████████████▊                                                                   | 1/5 [00:18<01:13, 18.37s/it]

Epoch: 2 | train_loss: 0.8975 | train_acc: 0.6562 | test_loss: 0.7837 | test_acc: 0.8561


 40%|█████████████████████████████████▌                                                  | 2/5 [00:36<00:54, 18.14s/it]

...

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:54<00:36, 18.21s/it]

Epoch: 4 | train_loss: 0.6770 | train_acc: 0.8516 | test_loss: 0.6698 | test_acc: 0.8049


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:11<00:17, 17.83s/it]

Epoch: 5 | train_loss: 0.7065 | train_acc: 0.7188 | test_loss: 0.6746 | test_acc: 0.7737


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:29<00:00, 17.84s/it]


In [13]:
results

{'train_loss': [1.0923891514539719,
  0.8974885493516922,
  0.8037762567400932,
  0.6769649833440781,
  0.7064977586269379],
 'train_acc': [0.3984375, 0.65625, 0.74609375, 0.8515625, 0.71875],
 'test_loss': [0.9132375717163086,
  0.7837276458740234,
  0.672258714834849,
  0.6698037981987,
  0.6745620767275492],
 'test_acc': [0.5397727272727273,
  0.8560606060606061,
  0.8863636363636364,
  0.8049242424242425,
  0.7736742424242425]}

## 5. View our model's results in TensorBoard

In [14]:
# %load_ext tensorboard
# %tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 5792), started 20:23:02 ago. (Use '!kill 5792' to kill it.)

## 6. Create a helper function to build `SummaryWriter()` instances

In [15]:
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
    from datetime import datetime
    import os
    timestamp = datetime.now().strftime("%Y-%m-%d")
    if extra:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [16]:
from typing import Dict, List
from tqdm import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        print(f"Epoch: {epoch+1} | train loss: {train_loss:.4f} | train acc: {train_acc:.4f} | test loss: {test_loss:.4f} | test acc: {test_acc:.4f}")
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        if writer:
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                                global_step=epoch)
            writer.close()
        else:
            pass
    return results

## 7. Setting up series of modelling experiments

In [17]:
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")
data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data\pizza_steak_sushi directory exists. skipping download...
[INFO] Did not find data\pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


In [18]:
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"
test_dir = data_10_percent_path / "test"
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: data\pizza_steak_sushi\train
Training directory 20%: data\pizza_steak_sushi_20_percent\train
Testing directory: data\pizza_steak_sushi\test


In [19]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])
simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [21]:
BATCH_SIZE = 32
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloader(train_dir=train_dir_10_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE
                                                                                         )
train_dataloader_20_percent, _,_ = data_setup.create_dataloader(train_dir=train_dir_20_percent,
                                                                 test_dir=test_dir,
                                                                 transform=simple_transform,
                                                                 batch_size=BATCH_SIZE
                                                                )
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(test_dataloader)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 3 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


In [22]:
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to C:\Users\Glen Dsilva/.cache\torch\hub\checkpoints\efficientnet_b2_rwightman-c35c1473.pth
100%|█████████████████████████████████████████████████████████████████████████████| 35.2M/35.2M [00:30<00:00, 1.21MB/s]


In [23]:
summary(model=effnetb2, 
        input_size=(32, 3, 224, 224), 
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1408, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [24]:
out_features = len(class_names)

def create_effnetb0(out_features: int) -> torch.nn.Module:
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(device)
    for param in model.features.parameters():
        param.requires_grad = False
    set_seeds()
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=out_features)
    ).to(device)
    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb2(out_features: int) -> torch.nn.Module:
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)
    for param in model.features.parameters():
        param.requires_grad = False
    set_seeds()
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=out_features)
    ).to(device)
    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

In [25]:
effnetb0 = create_effnetb0(out_features)

[INFO] Created new effnetb0 model.


In [26]:
effnetb2 = create_effnetb2(out_features)

[INFO] Created new effnetb2 model.


In [27]:
num_epochs = [5, 10]
models = ["effnetb0", "effnetb2"]
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [31]:
from going_modular.utils import save_model
set_seeds()
experiment_number = 0
for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number +=1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  
            if model_name == "effnetb0":
                model = create_effnetb0(out_features)
            else:
                model = create_effnetb2(out_features)
            loss_fn = nn.CrossEntropyLoss()
            optmizer = torch.optim.Adam(params = model.parameters(), lr=0.001)
            train(model=model,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    epochs=epochs,
                    device=device,
                    writer=create_writer(experiment_name=dataloader_name,
                                        model_name=model_name,
                                        extra=f"{epochs}_epochs"))
            save_filepath=f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                        target_dir="models",
                        model_name=save_filepath)
            print("-"*50+"\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-10-01\data_10_percent\effnetb0\5_epochs...


 20%|████████████████▊                                                                   | 1/5 [00:15<01:01, 15.33s/it]

Epoch: 1 | train loss: 1.1178 | train acc: 0.4062 | test loss: 1.0976 | test acc: 0.3409


 40%|█████████████████████████████████▌                                                  | 2/5 [00:30<00:45, 15.14s/it]

Epoch: 2 | train loss: 1.1203 | train acc: 0.2852 | test loss: 1.0919 | test acc: 0.3201


...

Epoch: 8 | train loss: 1.1297 | train acc: 0.2812 | test loss: 1.0898 | test acc: 0.3286


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:32<00:16, 16.99s/it]

Epoch: 9 | train loss: 1.1202 | train acc: 0.3250 | test loss: 1.0912 | test acc: 0.3485


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:50<00:00, 17.07s/it]

Epoch: 10 | train loss: 1.1216 | train acc: 0.3208 | test loss: 1.0962 | test acc: 0.3182
[INFO] Saving model to: models\07_effnetb2_data_20_percent_10_epochs.pth
--------------------------------------------------



In [37]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 5792), started 21:30:05 ago. (Use '!kill 5792' to kill it.)